In [ ]:
import pandas as pd
jigger_follow_ups_clean = pd.read_csv("codeclan/jigger_follow_ups_clean.csv")
jigger_details_clean = pd.read_csv("codeclan/jigger_details_clean.csv")
reasons = pd.read_csv("codeclan/joining_reasons.csv")
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:

reasons.rename(columns = {"id" : "userID"}, inplace = True)
jigger_lm = reasons

In [ ]:
#jigger_follow_ups_clean = jigger_follow_ups_clean.drop(jigger_follow_ups_clean.columns.difference(['past_week_30_final', 'userID']), 1).drop_duplicates()

In [ ]:
#jigger_follow_ups_clean

In [ ]:
#jigger_lm = jigger_follow_ups_clean.dropna(subset=['past_week_30_final'])

In [ ]:
#jigger_lm.shape

In [ ]:
#jigger_lm = pd.merge(jigger_lm, reasons, how='left', on=['userID'])

In [ ]:
jigger_lm.columns

In [ ]:
jigger_lm['simd_d'] = jigger_lm.simd_d.astype(str)

In [ ]:
jigger_lm['percentage_growth_act_days'] =  (((jigger_lm['past_week_30_final'] - jigger_lm['past_week_30_initial'] ) / jigger_lm['past_week_30_initial'])*100).round(0)
jigger_lm = jigger_lm[(jigger_lm['past_week_30_final'] != 0) & (jigger_lm['past_week_30_initial'] != 0)]                                                    

In [ ]:
jigger_lm.columns

In [ ]:
jigger_lm = jigger_lm.drop(['past_week_30_initial', 'past_week_30_final', 'currentlyRegularActiveFinal', 'userID'], axis=1).drop_duplicates()

##### Dataset ready

In [ ]:
import pandas_profiling
pandas_profiling.ProfileReport(jigger_lm)

In [ ]:
#get dummies
jigger_lm = pd.get_dummies(jigger_lm, drop_first = True)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
jigger_lm.columns

In [ ]:
y = jigger_lm['percentage_growth_act_days']
X = jigger_lm.drop('percentage_growth_act_days',axis=1)

In [ ]:
model = LinearRegression()
model.fit(X,y)

In [ ]:
model.score(X, y)
# 0.35 - not amazing - our model wont be great at predicting our target

In [ ]:
coefficients = pd.DataFrame({
    "Variable": X.columns.values,
    "Coefficient" : model.coef_
})
coefficients

In [ ]:
coefficients = coefficients[coefficients['Coefficient'] <=0.05]
coefficients = coefficients[coefficients['Coefficient'] >=0.05]
coefficients

In [ ]:
#now for p values
import statsmodels.api as sm
X_i = sm.add_constant(X)
X_i.head()

In [ ]:
sm_model = sm.OLS(y, X_i).fit()
print(sm_model.summary())

In [ ]:
# Certain schemes and local authority areas seemed good predictors of best percentage increase 
#Also, if you were already currently active, you lived ub a 2nd or 6th decile SIMD area, your aged 35 - 44 or aged 65 - 74
#Any of our other variables didnt provide meaningful difference - surprisingly, even thos elike health problem?